In [3]:
# from langsmith.wrappers.openai_wrapper import openai

In [4]:
# function_call_gets_no_tokens.py

from typing import Any
from uuid import UUID

from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import LLMChain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate

from pydantic import BaseModel, Field


class CustomCallbackHandler(BaseCallbackHandler):
    def on_llm_new_token(
        self,
        token: str,
        *,
        run_id: UUID,
        parent_run_id: UUID | None = None,
        **kwargs: Any,
    ) -> Any:
        print(f"'{token}'")  # wrap token in single quotes for visibility

        return super().on_llm_new_token(
            token, run_id=run_id, parent_run_id=parent_run_id, **kwargs
        )


class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


chat_model = ChatOpenAI(
    streaming=True,
    callbacks=[CustomCallbackHandler()],
)

joke_parser = PydanticOutputParser(pydantic_object=Joke)
prompt_template = "Tell me a joke about {subject}.\n\n{format_instructions}"
chat_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["subject"],
    partial_variables={"format_instructions": joke_parser.get_format_instructions()},
)


def chain_as_normal_call(subject):
    chain = LLMChain(llm=chat_model, prompt=chat_prompt)
    return chain(subject)


def chain_as_function_call(subject):
    chain = create_structured_output_chain(Joke, llm=chat_model, prompt=chat_prompt)
    return chain(subject)


print("Tokens received as part of a normal call:")
normal_joke = chain_as_normal_call("frogs")
print(normal_joke)
print()

Tokens received as part of a normal call:
''
'{
'
' '
' "'
'setup'
'":'
' "'
'Why'
' did'
' the'
' frog'
' go'
' to'
' the'
' bank'
'?",
'
' '
' "'
'p'
'unch'
'line'
'":'
' "'
'To'
' get'
' a'
' new'
' pad'
'!"
'
'}'
''
{'subject': 'frogs', 'text': '{\n  "setup": "Why did the frog go to the bank?",\n  "punchline": "To get a new pad!"\n}'}



In [13]:
from huggingface_hub import notebook_login

In [19]:
notebook_login()

In [22]:
from langchain import HuggingFaceHub

from langchain import PromptTemplate, LLMChain
import asyncio

question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm: HuggingFaceHub = HuggingFaceHub(
					repo_id="tiiuae/falcon-7b",
                    # repo_id="Qwen/Qwen-7B-Chat",
					huggingfacehub_api_token="MY-TOKEN",
					model_kwargs={
						"temperature": 0.5,
						 "max_length": 64
					}
				)

llm_chain = LLMChain(prompt=prompt, llm=llm)

async def run_chain():
    result = await llm_chain.arun(question)
    print('restult with load_qua_chain is....', result)

await run_chain()

ValueError: Error raised by inference API: Authorization header is correct, but the token seems invalid

In [5]:
print("Tokens not received as part of a function call:")
function_joke = chain_as_function_call("frogs")
print(function_joke)
print()

Tokens not received as part of a function call:
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
''
{'subject': 'frogs', 'function': Joke(setup='Why are frogs so happy?', punchline='Because they eat whatever bugs them!')}

